In [5]:
import warnings
warnings.filterwarnings("ignore")
import sys 
sys.path.append("../src")


import os
import pandas as pd

## Tracking

#### Load Tracking Data 

In [6]:
data_dir = "../data/v2"
exp = "a"
pos = 1
tracking_path = os.path.join(data_dir, f"tracking/position/{exp}{pos}.csv")
tracking = pd.read_csv(tracking_path, engine='python', index_col=0)
tracking

,blue_x,blue_y,yellow_x,yellow_y,focal_x,focal_y,blue_vx,blue_vy,yellow_vx,yellow_vy,focal_vx,focal_vy,missing_blue,missing_yellow,B2F,Y2F
frame,,,,,,,,,,,,,,,,
0,203.0,221.0,94.0,493.0,77.0,393.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0
1,206.0,227.0,93.0,494.0,77.0,393.0,3.0,6.0,-1.0,1.0,0.0,0.0,0,0,0,0
2,206.0,229.0,92.0,494.0,78.0,393.0,0.0,2.0,-1.0,0.0,1.0,0.0,0,0,0,0
3,208.0,231.0,97.0,493.0,78.0,394.0,2.0,2.0,5.0,-1.0,0.0,1.0,0,0,0,0
4,208.0,231.0,97.0,493.0,78.0,394.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35996,207.0,611.0,206.0,661.0,454.0,116.0,0.0,0.0,-1.0,0.0,0.0,0.0,0,1,0,0
35997,207.0,611.0,207.0,660.0,454.0,116.0,0.0,0.0,1.0,-1.0,0.0,0.0,0,1,0,0
35998,206.0,611.0,207.0,660.0,454.0,116.0,-1.0,0.0,0.0,0.0,0.0,0.0,0,1,0,0


#### Kalman Filter

```python

In [11]:
import pandas as pd
import numpy as np
from filterpy.kalman import KalmanFilter

def kalman_filter_2d(df, x_col, y_col, dt=1.0, process_var=1e-2, meas_var=1.0):
    kf = KalmanFilter(dim_x=4, dim_z=2)
    
    # State transition matrix
    kf.F = np.array([[1, 0, dt, 0],
                     [0, 1, 0, dt],
                     [0, 0, 1,  0],
                     [0, 0, 0,  1]])
    
    # Measurement function (we observe position only)
    kf.H = np.array([[1, 0, 0, 0],
                     [0, 1, 0, 0]])
    
    # Uncertainty in initial state
    kf.P *= 100.
    
    # Process noise
    kf.Q = process_var * np.eye(4)
    
    # Measurement noise
    kf.R = meas_var * np.eye(2)

    positions = df[[x_col, y_col]].values
    valid = ~np.isnan(positions).any(axis=1)

    # Initialize with first valid measurement
    first_idx = np.argmax(valid)
    kf.x = np.array([*positions[first_idx], 0, 0], dtype=float)

    xs, ys, vxs, vys = [], [], [], []

    for i in range(len(df)):
        if valid[i]:
            z = positions[i]
            kf.predict()
            kf.update(z)
        else:
            # predict only (no update if position is missing)
            kf.predict()
        
        xs.append(kf.x[0])
        ys.append(kf.x[1])
        vxs.append(kf.x[2])
        vys.append(kf.x[3])
    
    return xs, ys, vxs, vys

def smooth_all_agents(df, fps=30, process_var=1e-2, meas_var=1.0):
    df = df.copy()

    for agent in ['blue', 'yellow', 'focal']:
        x_s, y_s, vx_s, vy_s = kalman_filter_2d(df, f'{agent}_x', f'{agent}_y', 
                                                dt=1/fps, 
                                                process_var=process_var, 
                                                meas_var=meas_var)
        df[f'{agent}_x'] = x_s
        df[f'{agent}_y'] = y_s
        df[f'{agent}_vx'] = vx_s
        df[f'{agent}_vy'] = vy_s

    return df


In [12]:
tracking_smooth = smooth_all_agents(tracking, fps=30, process_var=1e-2, meas_var=1.0)
tracking_smooth

,blue_x,blue_y,yellow_x,yellow_y,focal_x,focal_y,blue_vx,blue_vy,yellow_vx,yellow_vy,focal_vx,focal_vy,missing_blue,missing_yellow,B2F,Y2F,B2F_dist,Y2F_dist
frame,,,,,,,,,,,,,,,,,,
0,203.000000,221.000000,94.000000,493.000000,77.000000,393.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0,0,0,0,213.213508,101.434708
1,204.580425,224.160850,93.473192,493.526808,77.000000,393.000000,4.773985,9.547970,-1.591328,1.591328,0.000000,0.000000,0,0,0,0,210.230826,102.259474
2,205.278908,226.413625,92.812459,493.759636,77.427905,393.000000,8.194328,21.815867,-5.445048,2.731443,2.713606,0.000000,0,0,0,0,208.038458,101.965681
3,206.568105,228.742625,94.444387,493.497596,77.718270,393.415063,15.979316,34.088864,8.449422,0.026090,4.245326,3.180209,0,0,0,0,208.492206,100.806746
4,207.473707,230.343880,95.669146,493.291731,77.917936,393.719703,18.783107,37.584300,15.539463,-1.528087,4.682517,4.673473,0,0,0,0,208.492206,100.806746
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35996,207.351332,611.865832,209.277993,661.593651,453.984897,116.000000,-0.424273,1.191330,5.400493,0.636947,-0.082908,0.000001,0,1,0,0,553.203398,598.772912
35997,207.296362,611.795898,209.160389,661.419350,453.984297,116.000000,-0.455884,1.106435,5.170053,0.485550,-0.081233,0.000001,0,1,0,0,553.203398,597.448743
35998,207.126040,611.731945,209.050273,661.261717,453.983818,116.000000,-0.575995,1.028361,4.951359,0.350968,-0.079507,0.000001,0,1,0,0,553.650612,597.448743


#### Extract Useful Variables

In [8]:
data_dir = "../data/v2"
exp = "a"
pos = 1
tracking_path = os.path.join(data_dir, f"tracking/position/{exp}{pos}.csv")
tracking = pd.read_csv(tracking_path, engine='python', index_col=0)
def euclidean_distance(row, agent1="blue", agent2="focal"):
    return ((row[f"{agent1}_x"] - row[f"{agent2}_x"]) ** 2 +
            (row[f"{agent1}_y"] - row[f"{agent2}_y"]) ** 2) ** 0.5
tracking["B2F_dist"] = tracking.apply(
    lambda row: euclidean_distance(row, agent1="blue", agent2="focal"), axis=1)
tracking["Y2F_dist"] = tracking.apply(
    lambda row: euclidean_distance(row, agent1="yellow", agent2="focal"), axis=1)

tracking



,blue_x,blue_y,yellow_x,yellow_y,focal_x,focal_y,blue_vx,blue_vy,yellow_vx,yellow_vy,focal_vx,focal_vy,missing_blue,missing_yellow,B2F,Y2F,B2F_dist,Y2F_dist
frame,,,,,,,,,,,,,,,,,,
0,203.0,221.0,94.0,493.0,77.0,393.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,213.213508,101.434708
1,206.0,227.0,93.0,494.0,77.0,393.0,3.0,6.0,-1.0,1.0,0.0,0.0,0,0,0,0,210.230826,102.259474
2,206.0,229.0,92.0,494.0,78.0,393.0,0.0,2.0,-1.0,0.0,1.0,0.0,0,0,0,0,208.038458,101.965681
3,208.0,231.0,97.0,493.0,78.0,394.0,2.0,2.0,5.0,-1.0,0.0,1.0,0,0,0,0,208.492206,100.806746
4,208.0,231.0,97.0,493.0,78.0,394.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,208.492206,100.806746
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35996,207.0,611.0,206.0,661.0,454.0,116.0,0.0,0.0,-1.0,0.0,0.0,0.0,0,1,0,0,553.203398,598.772912
35997,207.0,611.0,207.0,660.0,454.0,116.0,0.0,0.0,1.0,-1.0,0.0,0.0,0,1,0,0,553.203398,597.448743
35998,206.0,611.0,207.0,660.0,454.0,116.0,-1.0,0.0,0.0,0.0,0.0,0.0,0,1,0,0,553.650612,597.448743
